In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

In [ ]:
transform = transforms.ToTensor()
train_data = datasets.FashionMNIST(root='../data/cnn_files/', train=True, download=True, transform=transform)
test_data = datasets.FashionMNIST(root='../data/cnn_files/', train=False, download=True, transform=transform)
class_names = ['T-shirt', 'Trouser', 'Sweater', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Boot']

In [ ]:
train_data

In [ ]:
test_data

In [ ]:
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)
test_loader = DataLoader(test_data, batch_size=10, shuffle=False)

In [ ]:
for images, labels in train_loader:
    break

In [ ]:
print(labels)
print([class_names[index] for index in labels])
im = make_grid(images, nrow=10)

plt.figure(figsize=(12, 4))
plt.imshow(np.transpose(im.numpy(),(1,2,0)))

In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=3, stride=1, padding='same')
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=3,stride=1, padding='same')
        self.fc1 = nn.Linear(in_features=7*7*16, out_features=300)
        self.fc2 = nn.Linear(in_features=300, out_features=100)
        self.fc3 = nn.Linear(in_features=100, out_features=10)
        
    def forward(self, X):
        X = F.relu(self.conv1(X))
        X = F.max_pool2d(X, kernel_size=2, stride=2)
        X = F.relu(self.conv2(X))
        X = F.max_pool2d(X, kernel_size=2, stride=2)
        X = X.view(-1, 7*7*16)
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X = F.log_softmax(self.fc3(X), dim=1)
        
        return X

In [ ]:
torch.manual_seed(101)
model = ConvNet()
model

In [ ]:
for param in model.parameters():
    print(param.numel())

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
import time
start_time = time.time()

epoch = 5
train_losses_list = []
test_losses_list = []
train_corrects_list = []
test_corrects_list = []

for i in range(epoch):
    train_epoch_corrects = 0
    test_epoch_corrects = 0
    
    for batch_num, (X_train, y_train) in enumerate(train_loader):
        batch_num += 1
        
        y_pred = model.forward(X_train)
        loss = criterion(y_pred, y_train)
        
        predicted = torch.max(y_pred.data, dim=1)[1]
        batch_corrects = (predicted == y_train).sum()
        train_epoch_corrects += batch_corrects
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch_num % 2000 == 0:
            acc = (train_epoch_corrects / (batch_num*10))*100
            print(f'epoch:{i}, batch:{batch_num}, loss:{loss}, acc:{acc}')
        
    train_losses_list.append(loss)
    train_corrects_list.append(train_epoch_corrects)
    
    with torch.no_grad():
        for batch_num, (X_test, y_test) in enumerate(test_loader):
            batch_num += 1
            
            y_eval = model.forward(X_test)

            predicted = torch.max(y_eval.data, dim=1)[1]
            batch_corrects = (predicted == y_test).sum()
            test_epoch_corrects += batch_corrects
            
        loss = criterion(y_eval, y_test)
        test_losses_list.append(loss)
        test_corrects_list.append(test_epoch_corrects)

duration = time.time() - start_time
print(f'Total time is {duration/60} mins')

In [ ]:
torch.save(model.state_dict(), '../models/my_fashionmnist_cnn_1.pt')

In [ ]:
plt.plot(train_losses_list, label='train loss')
plt.plot(test_losses_list, label='validation loss')
plt.title('loss at epoch')
plt.legend()

In [ ]:
plt.plot([(t.item()/60000)*100 for t in train_corrects_list], label='train accuracy')
plt.plot([(t.item()/10000)*100 for t in test_corrects_list], label='validation accuracy')
plt.title('Accuracy at the end of each epoch')
plt.legend()

In [ ]:
all_test_data = DataLoader(test_data, batch_size=10000, shuffle=False)
with torch.no_grad():
    for X_test, y_test in all_test_data:
        y_eval = model.forward(X_test)
        predicted = torch.max(y_eval.data, dim=1)[1]
        batch_corrects = (predicted == y_test).sum()
print(f'acc = {batch_corrects}/{10000} = {(batch_corrects/10000)*100}')